## Decision Trees Implementation
Uses Information Gain as the measurement to be split


In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [2]:
data = datasets.load_iris

In [3]:
data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
names = data['target_names']

In [6]:
X = data['data']
X = np.array(X, dtype = int)
Y = data['target']

In [7]:
data_ = pd.DataFrame(X)
data_.columns = data['feature_names']
data_['Output'] = Y

In [8]:
data['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [9]:
data_.isna().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
Output               0
dtype: int64

In [11]:
# Define Entropy and Information Gain

In [12]:
def entropy(col):
    
    counts = np.unique(col,return_counts=True)
    N = float(col.shape[0])
    
    ent = 0.0
    
    for ix in counts[1]:
        p  = ix/N
        ent += (-1.0*p*np.log2(p))
    
    return ent

In [13]:
def information_gain(x_data,key,val):
    left,right = divide_data(x_data,key,val)
    
    #% of total samples are on left and right
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    
    #All examples come to one side!
    if left.shape[0] == 0 or right.shape[0] ==0:
        return -1000000 #Min Information Gain
    
    i_gain = entropy(x_data.Output) - (l*entropy(left.Output)+r*entropy(right.Output))
    return i_gain


In [14]:
def divide_data(x_data,key,val):
    #Work with Pandas Data Frames
    x_right = pd.DataFrame([],columns=x_data.columns)
    x_left = pd.DataFrame([],columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val_ = x_data[key].loc[ix]
        
        if val_ > val:
            x_right = x_right.append(x_data.loc[ix])
        else:
            x_left = x_left.append(x_data.loc[ix])
            
    return x_left,x_right


In [15]:
def find_count(X_train):                                   ###maintain the count of 0,1 and 2 for respectively
        count = []
        count.append(X_train[X_train['Output'] == 0].shape[0])
        count.append(X_train[X_train['Output'] == 1].shape[0])
        count.append(X_train[X_train['Output'] == 2].shape[0])
        return count

In [16]:
class DecisionTree:
    
    #Constructor
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.key = None
        self.val = None
        self.count = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
    
    def train(self,X_train,names):
        
        features = ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
        info_gains = []
        
        for ix in features:
            i_gain = information_gain(X_train,ix,X_train[ix].mean())
            info_gains.append(i_gain)
            
        self.key = features[np.argmax(info_gains)]
        self.val = X_train[self.key].mean()
        print("Level " , self.depth)
        self.count = find_count(X_train)
        cnt = 0                            ##maintain to get if we have only 1 flower in this node
        for i in range(len(self.count)):
            if(self.count[i]):
                print("Count of " , names[i] , " = " , self.count[i])
                cnt += 1
        print("Current entropy = " , entropy(X_train.Output))
        if cnt != 1:
            print("Splitting on Tree Features ",self.key,"with information gain",np.argmax(info_gains))
        
        #Split Data
        data_left,data_right = divide_data(X_train,self.key,self.val)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
         
        #Truly a left node
        if cnt == 1:
            if X_train.Output.mean() >= 1.5:
                self.target = names[2]
            elif X_train.Output.mean() <= 0.5:
                self.target = names[0]
            else:
                self.target = names[1]
            print("Reached leaf Node")
            print()
            print()
            return
        
        #Stop earyly when depth >=max depth
        if(self.depth>=self.max_depth):
            if X_train.Output.mean() >= 1.5:
                self.target = names[2]
            elif X_train.Output.mean() <= 0.5:
                self.target = names[0]
            else:
                self.target = names[1]
            print("Max depth Reached")
            print()
            print()
            return
        
        print()
        print()
        
        #Recursive Case
        self.left = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left, names)
        
        self.right = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right, names)
        
        #You can set the target at every node
        if X_train.Output.mean() >= 1.5:
            self.target = names[2]
        elif X_train.Output.mean() <= 0.5:
            self.target = names[0]
        else:
            self.target = names[1]
        return    

In [17]:
dt = DecisionTree()     #making object

In [18]:
dt.train(data_, names)      ##training or building tree

C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\21

C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\21

C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel

C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_485

C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4

...

C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\

Level  3
Count of  virginica  =  28
Current entropy =  0.0


C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_485

Reached leaf Node




C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])
C:\Users\hp\AppData\Local\Temp\ipykernel_4852\2129404573.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[ix])


# DECISION TREE 
USING iNBUILD CLASSIFIER


In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.tree import plot_tree

In [22]:
clf=DecisionTreeClassifier()
clf.fit(X,Y)

DecisionTreeClassifier()

In [25]:
dot_data=export_graphviz(clf,out_file=None,feature_names=data.feature_names)
graph=pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("irisdatasetproject.pdf")

True

In [27]:
# importing required modules
import PyPDF2
  
# creating a pdf file object
pdfFileObj = open('irisdatasetproject.pdf', 'rb')
  
# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  
# printing number of pages in pdf file
print(pdfReader.numPages)
  
# creating a page object
pageObj = pdfReader.getPage(0)
  
# extracting text from page
print(pageObj.extractText())
  
# closing the pdf file object
pdfFileObj.close()

1
petal length (cm) <= 2.0gini = 0.667samples = 150value = [50, 50, 50]
gini = 0.0samples = 50value = [50, 0, 0]True
petal length (cm) <= 4.5gini = 0.5samples = 100value = [0, 50, 50]False
petal width (cm) <= 1.5gini = 0.198samples = 54value = [0, 48, 6] petal width (cm) <= 1.5gini = 0.083samples = 46value = [0, 2, 44]
sepal length (cm) <= 4.5gini = 0.171samples = 53value = [0, 48, 5]gini = 0.0samples = 1value = [0, 0, 1]
petal length (cm) <= 3.5gini = 0.5samples = 2value = [0, 1, 1] sepal length (cm) <= 5.5gini = 0.145samples = 51value = [0, 47, 4]
gini = 0.0samples = 1value = [0, 1, 0] gini = 0.0samples = 1value = [0, 0, 1] gini = 0.0samples = 25value = [0, 25, 0]sepal length (cm) <= 6.5gini = 0.26samples = 26value = [0, 22, 4]
sepal width (cm) <= 2.5gini = 0.269samples = 25value = [0, 21, 4]gini = 0.0samples = 1value = [0, 1, 0]
gini = 0.231samples = 15value = [0, 13, 2] gini = 0.32samples = 10value = [0, 8, 2]sepal length (cm) <= 6.5gini = 0.198samples = 18value = [0, 2, 16]gini = 